# Imports
The following packages will be used:

1. tensorflow
2. numpy
3. pprint

In [ ]:
%%capture
!pip install --upgrade wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, ELU, LeakyReLU, Flatten, Dense, Add, AveragePooling2D, GlobalAveragePooling2D
import pprint
pp = pprint.PrettyPrinter(indent=4)

import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

# Organism
An organism contains the following:

1. phase - This denotes which phase does the organism belong to
2. chromosome - A dictionary of genes (hyperparameters)
3. model - The `tf.keras` model corresponding to the chromosome
4. prevBestOrganism - The best organism in the previous **phase**

In [ ]:
options_phase0 = {
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'activation_type': [ReLU, ELU, LeakyReLU],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D],
    'include_skip': [True, False]
}

options = {
    'include_layer': [True, False],
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D],
    'include_skip': [True, False]
}

In [ ]:
class Organism:
    def __init__(self,
                 chromosome={},
                 phase=0,
                 prevBestOrganism=None):
        '''
        chromosome is a dictionary of genes
        phase is the phase that the individual belongs to
        prevBestOrganism is the best organism of the previous phase
        '''
        self.phase = phase
        self.chromosome = chromosome
        self.prevBestOrganism=prevBestOrganism
        if phase != 0:
            # In a later stage, the model is made by
            # attaching new layers to the prev best model
            self.last_model = prevBestOrganism.model
    
    def build_model(self):
        '''
        This is the function to build the keras model
        '''
        keras.backend.clear_session()
        inputs = Input(shape=(32,32,3))
        if self.phase != 0:
            # Slice the prev best model
            # Use the model as a layer
            # Attach new layer to the sliced model
            intermediate_model = Model(inputs=self.last_model.input,
                                       outputs=self.last_model.layers[-3].output)
            for layer in intermediate_model.layers:
                # To make the iteration efficient
                layer.trainable = False
            inter_inputs = intermediate_model(inputs)
            x = Conv2D(filters=self.chromosome['a_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['a_filter_size'],
                       use_bias=self.chromosome['a_include_BN'])(inter_inputs)
            # This is to ensure that we do not randomly chose anothere activation
            self.chromosome['activation_type'] = self.prevBestOrganism.chromosome['activation_type']
        else:
            # For PHASE 0 only
            # input layer
            x = Conv2D(filters=self.chromosome['a_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['a_filter_size'],
                       use_bias=self.chromosome['a_include_BN'])(inputs)
        if self.chromosome['a_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)
        if self.chromosome['include_pool']:
            x = self.chromosome['pool_type'](strides=(1,1),
                                             padding='same')(x)
        if self.phase != 0 and self.chromosome['include_layer'] == False:
            # Except for PHASE0, there is a choice for
            # the number of layers that the model wants
            if self.chromosome['include_skip']:
                y = Conv2D(filters=self.chromosome['a_output_channels'],
                           kernel_size=(1,1),
                           padding='same')(inter_inputs)
                x = Add()([y,x])
            x = GlobalAveragePooling2D()(x)
            x = Dense(10, activation='softmax')(x)
        else:
            # PHASE0 or no skip
            # in the tail
            x = Conv2D(filters=self.chromosome['b_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['b_filter_size'],
                       use_bias=self.chromosome['b_include_BN'])(x)
            if self.chromosome['b_include_BN']:
                x = BatchNormalization()(x)
            x = self.chromosome['activation_type']()(x)
            if self.chromosome['include_skip']:
                y = Conv2D(filters=self.chromosome['b_output_channels'],
                           padding='same',
                           kernel_size=(1,1))(inputs)
                x = Add()([y,x])
            x = GlobalAveragePooling2D()(x)
            x = Dense(10, activation='softmax')(x)
        self.model = Model(inputs=[inputs], outputs=[x])
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
    def fitnessFunction(self,
                        train_ds,
                        test_ds,
                        generation_number):
        '''
        This function is used to calculate the
        fitness of an individual.
        '''
        # wandb.init()
        

        wandb.init(project="vlga",
                   group='KAGp{}'.format(self.phase),
                   job_type='g{}'.format(generation_number),settings=wandb.Settings(start_method="thread"))
        self.model.fit(train_ds,
                       epochs=3,
                       callbacks=[WandbCallback()],
                       verbose=0)
        _, self.fitness = self.model.evaluate(test_ds,
                                              verbose=0)
    def crossover(self,
                  partner,
                  generation_number):
        '''
        This function helps in making children from two
        parent individuals.
        '''
        child_chromosome = {}
        endpoint = np.random.randint(low=0, high=len(self.chromosome))
        for idx, key in enumerate(self.chromosome):
            if idx <= endpoint:
                child_chromosome[key] = self.chromosome[key]
            else:
                child_chromosome[key] = partner.chromosome[key]
        child = Organism(chromosome= child_chromosome, phase=self.phase, prevBestOrganism=self.prevBestOrganism)
        child.build_model()
        child.fitnessFunction(train_ds,
                              test_ds,
                              generation_number=generation_number)
        return child
    
    def mutation(self, generation_number):
        '''
        One of the gene is to be mutated.
        '''
        index = np.random.randint(0, len(self.chromosome))
        key = list(self.chromosome.keys())[index]
        if  self.phase != 0:
            self.chromosome[key] = options[key][np.random.randint(len(options[key]))]
        else:
            self.chromosome[key] = options_phase0[key][np.random.randint(len(options_phase0[key]))]
        self.build_model()
        self.fitnessFunction(train_ds,
                             test_ds,
                             generation_number=generation_number)
    
    def show(self):
        '''
        Util function to show the individual's properties.
        '''
        pp.pprint(self.chromosome)

def random_hyper(phase):
    if phase == 0:
        return {
        'a_filter_size': options_phase0['a_filter_size'][np.random.randint(len(options_phase0['a_filter_size']))],
        'a_include_BN': options_phase0['a_include_BN'][np.random.randint(len(options_phase0['a_include_BN']))],
        'a_output_channels': options_phase0['a_output_channels'][np.random.randint(len(options_phase0['a_output_channels']))],
        'activation_type': options_phase0['activation_type'][np.random.randint(len(options_phase0['activation_type']))],
        'b_filter_size': options_phase0['b_filter_size'][np.random.randint(len(options_phase0['b_filter_size']))],
        'b_include_BN': options_phase0['b_include_BN'][np.random.randint(len(options_phase0['b_include_BN']))],
        'b_output_channels': options_phase0['b_output_channels'][np.random.randint(len(options_phase0['b_output_channels']))],
        'include_pool': options_phase0['include_pool'][np.random.randint(len(options_phase0['include_pool']))],
        'pool_type': options_phase0['pool_type'][np.random.randint(len(options_phase0['pool_type']))],
        'include_skip': options_phase0['include_skip'][np.random.randint(len(options_phase0['include_skip']))]
        }
    else:
        return {
        'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
        'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
        'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
        'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
        'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
        'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
        'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
        'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))],
        'include_layer': options['include_layer'][np.random.randint(len(options['include_layer']))],
        'include_skip': options['include_skip'][np.random.randint(len(options['include_skip']))]
        }
        
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# Generation
This is a class that hold generations of models.

1. fitSurvivalRate - The amount of fit individuals we want in the next generation.
2. unfitSurvivalProb - The probability of sending unfit individuals
3. mutationRate - The mutation rate to change genes in an individual.
4. phase - The phase that the generation belongs to.
5. population_size - The amount of individuals that the generation consists of.
6. prevBestOrganism - The best organism (individual) is the last phase

In [ ]:
class Generation:
    def __init__(self,
                 fitSurvivalRate,
                 unfitSurvivalProb,
                 mutationRate,
                 phase,
                 population_size,
                 prevBestOrganism):
        self.population_size = population_size
        self.population = []
        self.generation_number = 0
        self.mutationRate = mutationRate
        self.fitSurvivalRate = fitSurvivalRate
        self.unfitSurvivalProb = unfitSurvivalProb
        self.prevBestOrganism = prevBestOrganism
        self.phase = phase
        # creating the first population: GENERATION_0
        # can be thought of as the setup function
        for idx in range(self.population_size):
            org = Organism(chromosome=random_hyper(self.phase), phase=self.phase, prevBestOrganism=self.prevBestOrganism)
            org.build_model()
            org.fitnessFunction(train_ds,
                                test_ds,
                                generation_number=self.generation_number)
            self.population.append(org)

        # sorts the population according to fitness (high to low)
        self.sortModel()
        self.generation_number += 1

    def sortModel(self):
        '''
        sort the models according to the 
        fitness in descending order.
        '''
        fitness = [ind.fitness for ind in self.population]
        sort_index = np.argsort(fitness)[::-1]
        self.population = [self.population[index] for index in sort_index]

    def generate(self):
        '''
        Generate a new generation in the same phase
        '''
        number_of_fit = int(self.population_size * self.fitSurvivalRate)
        new_pop = self.population[:number_of_fit]
        for individual in self.population[number_of_fit:]:
            if np.random.rand() <= self.unfitSurvivalProb:
                new_pop.append(individual)
        for index, individual in enumerate(new_pop):
            if np.random.rand() <= self.mutationRate:
                new_pop[index].mutation(generation_number=self.generation_number)
        fitness = [ind.fitness for ind in new_pop]
        children=[]
        for idx in range(self.population_size-len(new_pop)):
            parents = np.random.choice(new_pop, replace=False, size=(2,))
            A=parents[0]
            B=parents[1]
            child=A.crossover(B, generation_number=self.generation_number)
            children.append(child)
        self.population = new_pop+children
        self.sortModel()
        self.generation_number+=1

    def evaluate(self, last=False):
        '''
        Evaluate the generation
        '''
        fitness = [ind.fitness for ind in self.population]
        
        wandb.log({'Best fitness': fitness[0]})
        wandb.log({'Average fitness': sum(fitness)/len(fitness)})
        self.population[0].show()
        if last:
            return self.population[0]

# Data
The data that is being used for this experiment is the CIFAR10.

The dataset has 60,000 images of dimensions 32,32,3. 

In [ ]:
# Load the training and testing set of CIFAR10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32')
X_train = X_train/255.

X_test = X_test.astype('float32')
X_test = X_test/255.

y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

# Create TensorFlow dataset
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
population_size = 10
number_generation = 3

fitSurvivalRate = 0.5
unfitSurvivalProb = 0.2
mutationRate = 0.1
number_of_phases = 5

prevBestOrganism = None

for phase in range(number_of_phases):
    # print("PHASE {}".format(phase))
    generation = Generation(fitSurvivalRate=fitSurvivalRate,
                            unfitSurvivalProb=unfitSurvivalProb,
                            mutationRate=mutationRate,
                            population_size=population_size,
                            phase=phase,
                            prevBestOrganism=prevBestOrganism)
    while generation.generation_number < number_generation:
        generation.generate()
        if generation.generation_number == number_generation:
            # Last generation is the phase
            # print('I AM THE BEST IN THE PHASE')
            prevBestOrganism = generation.evaluate(last=True)
            keras.utils.plot_model(prevBestOrganism.model, to_file='best.png')
            wandb.log({"best_model": [wandb.Image('best.png', caption="Best Model")]})
        else:
            generation.evaluate()

wandb: Currently logged in as: doer173 (use `wandb login --relogin` to force relogin)


epoch,2
loss,1.59462
accuracy,0.41896
_runtime,103
_timestamp,1618185931
_step,2


epoch,▁▅█
loss,█▃▁
accuracy,▁▆█
_runtime,▁▅█
_timestamp,▁▅█
_step,▁▅█


epoch,2
loss,1.64226
accuracy,0.40668
_runtime,23
_timestamp,1618185959
_step,2


epoch,▁▅█
loss,█▃▁
accuracy,▁▆█
_runtime,▁▅█
_timestamp,▁▅█
_step,▁▅█


...

epoch,▁▅█
loss,█▂▁
accuracy,▁▇█
_runtime,▁▄█
_timestamp,▁▄█
_step,▁▅█


epoch,2
loss,1.205
accuracy,0.56918
_runtime,168
_timestamp,1618198902
_step,2


epoch,▁▅█
loss,█▂▁
accuracy,▁▇█
_runtime,▁▄█
_timestamp,▁▄█
_step,▁▅█


epoch,2
loss,1.2181
accuracy,0.56736
_runtime,140
_timestamp,1618199054
_step,2


epoch,▁▅█
loss,█▂▁
accuracy,▁▇█
_runtime,▁▅█
_timestamp,▁▅█
_step,▁▅█
